# DO NOT TOUCH THIS!!!

In [1]:
import sys
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import urllib
from threading import Thread
from queue import Queue
import math
import requests
import time 
from datetime import timedelta
import pickle

In [2]:
def buildURL(baseURL):
    url = "?isSearchRequest=true&vc=Car&dam=0&con=USED&ambitCountry=DE"
    
    # explicitly specify categories to prevent trailers etc. from showing up
    url += "&categories=Cabrio"
    url += "&categories=OffRoad"
    url += "&categories=SmallCar"
    url += "&categories=EstateCar"
    url += "&categories=Limousine"
    url += "&categories=SportsCar"
    url += "&categories=Van"   
    
    return baseURL + url

In [3]:
def buildURLParameters(mileageFrom = -1,
                       mileageTo = -1,
                       firstRegistrationFrom = -1,
                       firstRegistrationTo = -1,
                       priceFrom = -1,
                       priceTo = -1,
                       powerFrom = -1,
                       powerTo = -1):
        
    parameters = ""
    if(mileageFrom >= 0):
        parameters += "&minMileage=" + repr(mileageFrom)
    if(mileageTo >= 0):
        parameters += "&maxMileage=" + repr(mileageTo)
    if(firstRegistrationFrom >= 0):
        parameters += "&minFirstRegistrationDate=" + repr(firstRegistrationFrom) + "-01-01"
    if(firstRegistrationTo >= 0):
        parameters += "&maxFirstRegistrationDate=" + repr(firstRegistrationTo) + "-12-31"
    if(priceFrom >= 0):
        parameters += "&minPrice=" + repr(priceFrom)
    if(priceTo >= 0):
        parameters += "&maxPrice=" + repr(priceTo)
        
    return parameters

In [4]:
relevantIDs = set()
dropped = 0
taskQueue = Queue(5000)

In [5]:
def getResultCount(url):
    nResults = -1
    while nResults == -1:
        try:
            response = requests.get(url)
            json = response.json()
            nResults = json['numResultsTotal']
        except Exception as e:
            # try again
            nResults = -1
    return nResults

In [6]:
def scrapeResultList(baseURL, nResults):
    pageCount = int(math.ceil(nResults / 20.0))

    if pageCount > 50:
        pageCount = 50
        
    for i in range(pageCount):
        try:
            url = urllib.urlopen(baseURL + "&pageNumber=" + repr(i + 1))
            url = url.read()
            soup = BeautifulSoup(url, "lxml")
            div_results = soup.find_all("div", class_="cBox-body cBox-body--resultitem")
            
            for div_result in div_results:
                relevantIDs.add(div_result.a["data-ad-id"])
                
            sys.stdout.write("\r#IDs: " + repr(len(relevantIDs)) + 
                             " / #Queue: " + repr(taskQueue.qsize()) + 
                             " / #Dropped: " + repr(dropped))
        except Exception as e:
            # try again
            i -= 1 

In [7]:
class ResultListScraper(Thread):
    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue
    def run(self):
        while True:
            try:
                baseURL, nResults = self.queue.get()
                scrapeResultList(baseURL, nResults)
            except Exception as e:
                print(e)
            finally:
                self.queue.task_done()

In [8]:
dropped = 0
class TaskGenerator(Thread):
    def __init__(self, mileageFrom, mileageTo, priceFrom, priceTo):
        Thread.__init__(self)
        self.mileageFrom = mileageFrom
        self.mileageTo = mileageTo
        self.priceFrom = priceFrom
        self.priceTo = priceTo
    def run(self):#1, 1501
        for mileage in range(self.mileageFrom, self.mileageTo): # mileage between 0 and 1500000, interval 1000
            parametersMileage = buildURLParameters(mileageFrom=(mileage-1)*1000,
                                                   mileageTo=mileage*1000)
            mileageSearchURL = baseSearchURL + parametersMileage
            mileageCountURL = baseCountURL + parametersMileage
            nResultsMileage = getResultCount(mileageCountURL)

            if nResultsMileage <= 1000:
                taskQueue.put((mileageSearchURL, nResultsMileage))
            else:
                for firstRegistration in range(1900, 2017): # first registration between 1900 and 2016, interval 1
                    parametersRegistration = buildURLParameters(firstRegistrationFrom=firstRegistration,
                                                                firstRegistrationTo=firstRegistration)
                    registrationSearchURL = mileageSearchURL + parametersRegistration
                    registrationCountURL = mileageCountURL + parametersRegistration
                    nResultsRegistration = getResultCount(registrationCountURL)

                    if nResultsRegistration <= 1000:
                        taskQueue.put((registrationSearchURL, nResultsRegistration))
                    else: #1,3001
                        for price in range(self.priceFrom, self.priceTo): # price between 0 and 300000, interval 100
                            parametersPrice = buildURLParameters(priceFrom=(price-1)*100,
                                                                 priceTo=price*100)
                            priceSearchURL = registrationSearchURL + parametersPrice
                            priceCountURL = registrationCountURL + parametersPrice
                            nResultsPrice = getResultCount(priceCountURL)

                            taskQueue.put((priceSearchURL, nResultsPrice))

                            if nResultsPrice > 1000:
                                taskQueue.put((priceSearchURL + "&sortOption.sortOrder=DESCENDING", nResultsPrice))

                                if nResultsPrice > 2000:
                                    global dropped
                                    dropped += (nResultsPrice - 2000)

In [ ]:
taskQueue = Queue(20000)
for x in range(2):
    resultListScraper = ResultListScraper(taskQueue)
    resultListScraper.daemon = True
    resultListScraper.start()

baseSearchURL = buildURL("http://suchen.mobile.de/fahrzeuge/auto")
baseCountURL = buildURL("http://suchen.mobile.de/fahrzeuge/count.json")

idScrapingStartTime = time.time()
print("Scraping started at " + time.asctime(time.gmtime(idScrapingStartTime)))

taskGenerators = []
taskGenerators.append(TaskGenerator(1, 50, 1, 1001))
taskGenerators.append(TaskGenerator(51, 100, 1, 1001))
taskGenerators.append(TaskGenerator(101, 850, 1, 1001))
taskGenerators.append(TaskGenerator(851, 1501, 1, 1001))

for taskGenerator in taskGenerators:
    taskGenerator.daemon = True
    taskGenerator.start()

for taskGenerator in taskGenerators:
    taskGenerator.join()
        
taskQueue.join()

idScrapingEndTime = time.time()
print("\nScraping finished at " + time.asctime(time.gmtime(idScrapingEndTime)))

Scraping started at Thu Dec  1 16:23:29 2016
#IDs: 4184 / #Queue: 2962 / #Dropped: 0

In [ ]:
print "Current baseline count: " + repr(getResultCount(baseCountURL))
print "Relevant ID count: " + repr(len(relevantIDs))
print "Start time: " + time.asctime(time.gmtime(idScrapingStartTime))
print "End time: " + time.asctime(time.gmtime(idScrapingEndTime))
print "Duration: " + str(timedelta(seconds=(idScrapingEndTime-idScrapingStartTime)))

In [ ]:
relevantIDfile = open('relevantIDs.pckl', 'w')
pickle.dump(relevantIDs, relevantIDfile)

In [ ]:
def extractValue(tag):
    if(len(tag) <= 0):
        return np.nan
    return tag[0].stripped_strings.next()

In [ ]:
def extractInformation(resultID):
    success = False    
    while not success:
        try:
            url = urllib.urlopen("http://suchen.mobile.de/fahrzeuge/details.html?id=" + str(resultID))
            url = url.read()
            success = True
        except Exception as e:
            print "\nException opening http://suchen.mobile.de/fahrzeuge/details.html?id=" + str(resultID) + ", retrying."
            success = False
            
    soup = BeautifulSoup(url, "lxml")
    
    # ID does not exist (yet / any longer)
    if len(soup.find_all("div", class_="cBox-body cBox-body--notification-error")) > 0:
        return None
    
    mileage = extractValue(soup.select("#rbt-mileage-v"))
    power = extractValue(soup.select("#rbt-power-v"))
    fuel = extractValue(soup.select("#rbt-fuel-v"))
    transmission = extractValue(soup.select("#rbt-transmission-v"))
    firstRegistration = extractValue(soup.select("#rbt-firstRegistration-v"))
    price = extractValue(soup.select("span.h3.rbt-prime-price"))
    
    return [resultID, mileage, power, fuel, transmission, firstRegistration, price]

In [ ]:
relevantIDsList = list(relevantIDs) # just for output
relevantIDCount = len(relevantIDsList)
testData = []

In [ ]:
class ResultScraper(Thread):
    def __init__(self, queue):
        Thread.__init__(self)
        self.queue = queue
    def run(self):
        while True:
            try:
                resultID = self.queue.get()
                testData.append(extractInformation(resultID))
            except Exception as e:
                print(e)
            finally:
                self.queue.task_done()

In [ ]:
taskQueue = Queue(500)
for x in range(3):
    worker = ResultScraper(taskQueue)
    worker.daemon = True
    worker.start()

resultScrapingStartTime = time.time()
    
for i in range(len(relevantIDsList)):
    sys.stdout.write("\rQueueing " + relevantIDsList[i] + 
                     " (" + repr((i+1)) + 
                     " / " + repr(relevantIDCount) + 
                     ", " + repr(((i + 1.0) / relevantIDCount) * 100) + "%)")
    taskQueue.put(relevantIDsList[i])
    
taskQueue.join()
resultScrapingEndTime = time.time()